In [6]:
import pandas as pd
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora, models
from gensim.utils import simple_preprocess
import gensim, spacy
from gensim.models.ldamulticore import LdaMulticore
import re
from gensim.models import Phrases
from gensim.models.phrases import Phraser
import numpy as np

In [7]:
from gsdmm import MovieGroupProcess

In [8]:
df = pd.read_csv("gsdmm_india_results.csv")
df

,Unnamed: 0,Text,translated,words_processed_noun,tokenised_stopremoved,Lemma_text,Topic,len
0,0,soniya mummy hoti toh covid me magic stick ghu...,soniya mummy hoti toh covid me magic stick ghu...,['death'],['death'],death,type 1,25
1,1,Vaccinate to protect yourself and your loved o...,Vaccinate to protect yourself and your loved o...,[],[],NaN,Other,12
2,2,is covid ki waja se buht saray logon ne apne p...,is covid ki waja se buht saray logon ne apne p...,"['hair', 'ply', 'vaccination']","['hair', 'ply']",hair ply,Other,37
3,3,deaths due covid what is vaccination rate what...,deaths due covid what is vaccination rate what...,"['death', 'couid', 'vaccination', 'rate', 'har...","['death', 'rate', 'harm', 'harm']",death rate harm harm,type 1,61
4,4,vaccination against covid- 19 is necessity tod...,vaccination against covid- 19 is necessity tod...,"['necessity', 'today', 'scenario', 'problem']","['necessity', 'scenario', 'problem']",necessity scenario problem,type 2,17
...,...,...,...,...,...,...,...,...
43458,43458,nature seems have already started working savi...,nature seems have already started working savi...,"['nature', 'generation', 'life', 'strengthen']","['nature', 'generation', 'life', 'strengthen']",nature generation life strengthen,type 1,32
43459,43459,"dear , we #india Indiahas helped this #world i...","dear , we #india Indiahas helped this #world i...","['world', 'kindnessmatter']","['world', 'kindnessmatter']",world kindnessmatter,type 4,23
43460,43460,our sincere request state central govt folded_...,our sincere request state central govt folded_...,"['request', 'state', 'govt', 'share']","['request', 'state', 'govt', 'share']",request state govt share,type 2,11
43461,43461,thanks your appreciation valuable feedback abo...,thanks your appreciation valuable feedback abo...,"['thank', 'appreciation', 'feedback']","['appreciation', 'feedback']",appreciation feedback,type 3,9


In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [10]:
tokenised_lemmatized = list(sent_to_words(df['tokenised_stopremoved']))

In [11]:
tokenised_lemmatized

[['death'],
 [],
 ['hair', 'ply'],
 ['death', 'rate', 'harm', 'harm'],
 ['necessity', 'scenario', 'problem'],
 ['population',
  'yrs',
  'offline',
  'class',
  'examination',
  'situation',
  'safety',
  'class',
  'examination'],
 ['way', 'millennium', 'peril', 'job', 'process'],
 ['symptom', 'example', 'number', 'death', 'population', 'life'],
 ['follower', 'pledge', 'service', 'humanity'],
 ['case', 'case', 'testing', 'taboo'],
 ['intellectual',
  'education',
  'leader',
  'health',
  'system',
  'pandemic',
  'nation',
  'service'],
 ['memory', 'post', 'post', 'forgetting', 'memory', 'couple'],
 ['agreement', 'student', 'travel', 'guideline'],
 ['system',
  'conversion',
  'appeasement',
  'health',
  'service',
  'infection',
  'rate',
  'death',
  'rate',
  'rate',
  'welfare',
  'isis'],
 ['personnel', 'unitedstate'],
 ['oppression', 'appointment', 'wife', 'worker', 'present', 'money'],
 ['oppression', 'appointment', 'wife', 'center', 'worker'],
 ['result',
  'effect',
  'exer

In [12]:
#Load the save classifier 
import pickle
model_saved = open("india_gsdmm.pickle", "rb") #binary read
model = pickle.load(model_saved)
model_saved.close()

In [15]:
scores = []
for a_tweet in tokenised_lemmatized:
    score = model.score(a_tweet)
    scores.append(score)

In [16]:
topicnames = ["Topic" + str(i) for i in range(6)]
docnames = ["Doc" + str(i) for i in range(len(tokenised_lemmatized))]
df_document_topic = pd.DataFrame(np.round(scores, 5), columns=topicnames, index=docnames)

In [17]:
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

In [18]:
# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
num_doc_to_show = 50  # change this to display the number of documents shown below
df_document_topics = df_document_topic.head(num_doc_to_show).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.705410,0.033440,0.216800,0.007070,0.028270,0.009000,0
Doc1,0.281180,0.051590,0.250530,0.105170,0.224350,0.087180,0
Doc2,0.229220,0.216840,0.075480,0.279850,0.180460,0.018150,3
Doc3,0.996220,0.000410,0.003190,0.000110,0.000050,0.000020,0
Doc4,0.483370,0.001770,0.493240,0.001710,0.016950,0.002960,2
Doc5,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1
Doc6,0.002320,0.005640,0.669830,0.004530,0.317610,0.000070,2
Doc7,0.998580,0.000420,0.000800,0.000000,0.000190,0.000000,0
Doc8,0.025350,0.000000,0.912890,0.000310,0.061440,0.000000,2
Doc9,0.228210,0.183190,0.010940,0.562850,0.004300,0.010500,3
